<a href="https://colab.research.google.com/github/Manwa94/Multi_Document_RAG_QA_Groq_LLAMA/blob/main/Multi_Document_RAG_QA_Groq_LLAMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-community==0.2.15 langchain-chroma==0.1.3 langchain-text-splitters==0.2.2 langchain-huggingface==0.0.3 langchain-groq==0.1.9 unstructured==0.15.0 unstructured[pdf]==0.15.0 nltk==3.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 204.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [1]:
!apt-get install poppler-utils
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (458 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [2]:
import os

from langchain_community.document_loaders import UnstructuredPDFLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [7]:
GROQ_API_KEY = "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"

In [8]:
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [10]:
loader = DirectoryLoader("data/", glob="./*.pdf", loader_cls=UnstructuredPDFLoader)
documents = loader.load()

In [11]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=500
)

text_chunks = text_splitter.split_documents(documents)

In [12]:
persist_directory = "doc_db"

In [13]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
vectorstore = Chroma.from_documents(
    documents=text_chunks,
    embedding=embedding,
    persist_directory=persist_directory
)

In [18]:
retriever = vectorstore.as_retriever()

In [20]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

In [21]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [25]:
query = "What does the document say about DELINEATING THE CLINICAL SYNDROME?"
response = qa_chain.invoke({"query":query})

In [26]:
print(response)

{'query': 'What does the document say about DELINEATING THE CLINICAL SYNDROME?', 'result': 'The document states that it is useful to divide Respiratory Tract Infections (RTIs) into those involving the upper and the lower tracts. \n\nThe clinical syndromes involving the upper tract include:\n\n1. Otitis media\n2. Mastoiditis\n3. Sinusitis\n4. Pharyngitis\n\nThe clinical syndromes involving the lower tract can be divided into:\n\n1. Tracheobronchitis\n2. Bronchiolitis\n3. Pneumonia\n\nIt also mentions that most of these conditions can exist in acute and chronic forms. \n\nAcute disease is usually caused by viral or bacterial infections.\n\nChronic disease is usually caused by:\n\n1. Fungi\n2. Slow-growing bacteria such as mycobacteria\n3. Bacteria adapted to persist in biofilms\n4. Occasional less common pathogens such as parasites\n\nChronic infections can also develop when structural changes occur as a result of:\n\n1. Recurrent or severe acute infections\n2. Surgical intervention\n3. 

In [27]:
print(response["result"])

The document states that it is useful to divide Respiratory Tract Infections (RTIs) into those involving the upper and the lower tracts. 

The clinical syndromes involving the upper tract include:

1. Otitis media
2. Mastoiditis
3. Sinusitis
4. Pharyngitis

The clinical syndromes involving the lower tract can be divided into:

1. Tracheobronchitis
2. Bronchiolitis
3. Pneumonia

It also mentions that most of these conditions can exist in acute and chronic forms. 

Acute disease is usually caused by viral or bacterial infections.

Chronic disease is usually caused by:

1. Fungi
2. Slow-growing bacteria such as mycobacteria
3. Bacteria adapted to persist in biofilms
4. Occasional less common pathogens such as parasites

Chronic infections can also develop when structural changes occur as a result of:

1. Recurrent or severe acute infections
2. Surgical intervention
3. Other processes that alter the structural integrity of the respiratory tract.


In [28]:
query = "What does the document say about Receptor-binding domain?"
response = qa_chain.invoke({"query":query})
print(response["result"])

The document states that Receptor-binding domain (RBD) MAbs (monoclonal antibodies) binding near the neck and shoulders of the RBD have either footprint or spatial overlap with ACE2, enabling them to block ACE2 binding and thus have the strongest neutralizing abilities against SARS-CoV-2.

It also mentions that Mabs with epitopes far from this area, lower on the flanks of the RBD, are usually much less potent.

Additionally, the document mentions that the RBD is involved in the binding of SARS-CoV-2 to the ACE2 receptor, and that mutations in the RBD can affect the ability of MAbs to bind to the virus.

The document also provides a figure (Figure 1b) that shows the RBD and other parts of the SARS-CoV-2 spike protein, including the SD1 on S1 and the stem-helix region and the fusion peptide in S2.

Finally, the document mentions that the RBD is the target of some MAbs, including LY-CoV1404 (bebtelovimab), 2–7, XGv265, XG005, AZD1061, MB.02, SP1–77, and 002-S21F2, which bind to the front 